<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# 1D Balsara Shock Tests

## This module provides initial data options for GRMHD evolutions.

**Notebook Status:** <font color='green'><b> Validated </b></font>

**Validation Notes:** This tutorial notebook has been confirmed to be self-consistent with its corresponding NRPy+ module, as documented [below](#code_validation). 

### NRPy+ Source Code for this module: [ShockTests_1D.py](../../edit/in_progress-ShockTests/ShockTests_1D.py)

## Introduction:

Define shocks.

Shocks can occur commonly in astrophysical contexts. Among other methods, shocks may be modeled using codes that solve the equations of magnetohydrodynamics. Here we will develop 1-3D shock tests for MHD codes. Within NRPy+, we will specifically use these tests to validate the NRPy+ version of IllinoisGRMHD. We will borrow the same tests used by the Spritz code for its validation, described [here](https://iopscience.iop.org/article/10.1088/1361-6382/ab8be8).

In this notebook we document and implement 1D shock tests commonly used within the literature, i.e. the Balsara shock tube tests, found in table 1 [here](https://iopscience.iop.org/article/10.1086/318941). These tests are performed in flat spacetime.


<a id='toc'></a>

# Table of Contents:
$$\label{toc}$$

This notebook is organized as follows

1. [Step 1](#initializenrpy): Import core NRPy+ modules and set NRPy+ parameters
1. [Step 2](#shock_tests): Balsara 1D Shock Tests
    1. [Step 2.a](#balsara1): Balsara 1
    1. [Step 2.b](#balsara2): Balsara 2
    1. [Step 2.c](#balsara3): Balsara 3
    1. [Step 2.d](#balsara4): Balsara 4
    1. [Step 2.e](#balsara5): Balsara 5
1. [Step 3](#BtoA): Converting to Vector Potential $A_i$
1. [Step 4](#mrhs): Mildly Relativistic Hydrodynamic Shock
1. [Step 5](#rhbw): Relativistic Hydrodynamic Blast Wave
1. [Step 6](#code_validation): Code Validation against `ShockTests_1D` NRPy+ Module
1. [Step 7](#latex_pdf_output): Output this notebook to $\LaTeX$-formatted PDF file

<a id='initializenrpy'></a>

# Step 1: Import core NRPy+ modules and set NRPy+ parameters \[Back to [top](#toc)\]
$$\label{initializenrpy}$$

Here, we will import the NRPy+ core modules, set the reference metric to Cartesian, and set commonly used NRPy+ parameters. We will also set up a parameter to determine what initial data is set up, although it won't do much yet.

In [1]:
# Step 0: Add NRPy's directory to the path
# https://stackoverflow.com/questions/16780014/import-file-from-parent-directory
import os,sys
nrpy_dir_path = os.path.join("..")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)

# Step 0.a: Import the NRPy+ core modules and set the reference metric to Cartesian
import sympy as sp               # SymPy: The Python computer algebra package upon which NRPy+ depends
import NRPy_param_funcs as par   # NRPy+: Parameter interface
import indexedexp as ixp         # NRPy+: Symbolic indexed expression (e.g., tensors, vectors, etc.) support
import reference_metric as rfm   # NRPy+: Reference metric support

par.set_parval_from_str("grid::DIM", 3)
DIM = par.parval_from_str("grid::DIM")

par.set_parval_from_str("reference_metric::CoordSystem","Cartesian")
rfm.reference_metric()

##### <a id='shock_tests'></a>

# Step 2: Balsara 1D Shock Tests \[Back to [top](#toc)\]
$$\label{shock_tests}$$

Each test defines the GRMHD primitive variables  as piece-wise functions, with left and right sides separated by a discontinuity or shock. These primitive variables are the fluid density $\rho$, pressure $p$, velocity $v^i$ and magnetic field $B^i$. For ease in transcriptions we follow [Balasra (2001)](https://iopscience.iop.org/article/10.1086/318941) and define these quantities in Cartesian coordinates. As stated by [Balasra (2001)](https://iopscience.iop.org/article/10.1086/318941), "the initial discontinuity was placed at the center of the computational domain", so we choose to place them at $x=0$.

To aid in speedy calculations, instead of using `if` statements to define these piece-wise functions, we use the `Min_Max_and_Piecewise_Expressions` NRPy+ module, which replaces `if` statements with absolute value calls. In C these are done using the `fabs` function.

How do we use `Min_Max_and_Piecewise_Expressions`???


##### <a id='balsara1'></a>

## Step 2.a: Balsara 1 \[Back to [top](#toc)\]
$$\label{balsara1}$$



\begin{align}
\rho &= \left \{ \begin{array}{lll} 1.0 & \mbox{if} & x < 0 \\
0.125 & \mbox{if} & x \geq 0 \end{array} \right. \\
p &= \left \{ \begin{array}{lll} 1.0 & \mbox{if} & x < 0 \\
0.1 & \mbox{if} & x \geq 0 \end{array} \right. \\
v^x &= v^y = v^z = 0.0, \\
B^x &= 0.5, \\
B^y &= \left \{ \begin{array}{lll} 1.0 & \mbox{if} & x < 0 \\
-1.0 & \mbox{if} & x \geq 0 \end{array} \right. \\
B^z &= 0.0
\end{align}

In [2]:
import Min_Max_and_Piecewise_Expressions as noif
from sympy import Rational as rl

def balsara1(x, bound = 0.0):
    vU = ixp.zerorank1()
    BU = ixp.zerorank1()

    rho_left = rl(1.0)
    rho_right = rl(1,8)
    rho = noif.coord_less_bound(x, bound)*rho_left \
        + noif.coord_geq_bound(x,bound)*rho_right

    press_left = rl(1.0)
    press_right = rl(1,10)
    press = noif.coord_less_bound(x, bound)*press_left \
          + noif.coord_geq_bound(x,bound)*press_right

    BU[0] = rl(1,2)

    By_left = rl(1.0)
    By_right = rl(-1.0)
    BU[1] = noif.coord_less_bound(x, bound)*By_left \
          + noif.coord_geq_bound(x,bound)*By_right

    return rho, press, vU, BU

##### <a id='balsara2'></a>

## Step 2.b: Balsara 2 \[Back to [top](#toc)\]
$$\label{balsara2}$$

\begin{align}
\rho &= 1.0 \\
p &= \left \{ \begin{array}{lll} 30.0 & \mbox{if} & x < 0 \\
1.0 & \mbox{if} & x \geq 0 \end{array} \right. \\
v^x &= v^y = v^z = 0.0, \\
B^x &= 5.0, \\
B^y &= B^z = \left \{ \begin{array}{lll} 6.0 & \mbox{if} & x < 0 \\
0.7 & \mbox{if} & x \geq 0 \end{array} \right. \\
\end{align}

In [3]:
def balsara2(x, bound = 0.0):
    vU = ixp.zerorank1()
    BU = ixp.zerorank1()

    rho = rl(1.0)

    press_left = rl(30.0)
    press_right = rl(1.0)
    press = noif.coord_less_bound(x, bound)*press_left \
          + noif.coord_geq_bound(x,bound)*press_right

    BU[0] = rl(1,2)

    By_left = rl(6.0)
    By_right = rl(7,10)
    BU[1] = BU[2] = noif.coord_less_bound(x, bound)*By_left \
                  + noif.coord_geq_bound(x,bound)*By_right

    return rho, press, vU, BU

##### <a id='balsara3'></a>

## Step 2.c: Balsara 3 \[Back to [top](#toc)\]
$$\label{balsara3}$$

\begin{align}
\rho &= 1.0 \\
p &= \left \{ \begin{array}{lll} 1000.0 & \mbox{if} & x < 0 \\
0.1 & \mbox{if} & x \geq 0 \end{array} \right. \\
v^x &= v^y = v^z = 0.0, \\
B^x &= 10.0, \\
B^y &= B^z = \left \{ \begin{array}{lll} 7.0 & \mbox{if} & x < 0 \\
0.7 & \mbox{if} & x \geq 0 \end{array} \right. \\
\end{align}

In [4]:
def balsara3(x, bound = 0.0):
    vU = ixp.zerorank1()
    BU = ixp.zerorank1()

    rho = rl(1.0)

    press_left = rl(1000.0)
    press_right = rl(1,10)
    press = noif.coord_less_bound(x, bound)*press_left \
          + noif.coord_geq_bound(x,bound)*press_right

    BU[0] = rl(10.)

    By_left = rl(7.0)
    By_right = rl(7,10)
    BU[1] = BU[2] = noif.coord_less_bound(x, bound)*By_left \
                  + noif.coord_geq_bound(x,bound)*By_right

    return rho, press, vU, BU

##### <a id='balsara4'></a>

## Step 2.d: Balsara 4 \[Back to [top](#toc)\]
$$\label{balsara4}$$

\begin{align}
\rho &= 1.0 \\
p &= 0.1 \\
v^x &= \left \{ \begin{array}{lll} 0.999 & \mbox{if} & x < 0 \\
-0.999 & \mbox{if} & x \geq 0 \end{array} \right. \\
v^y &= v^z = 0.0, \\
B^x &= 10.0, \\
B^y &= B^z = \left \{ \begin{array}{lll} 7.0 & \mbox{if} & x < 0 \\
-7.0 & \mbox{if} & x \geq 0 \end{array} \right. \\
\end{align}

In [5]:
def balsara4(x, bound = 0.0):
    vU = ixp.zerorank1()
    BU = ixp.zerorank1()

    rho = rl(1.0)

    press = rl(1,10)

    vx_left = rl(999,1000)
    vx_right = rl(-999,1000)
    vU[0] = noif.coord_less_bound(x, bound)*vx_left \
          + noif.coord_geq_bound(x,bound)*vx_right

    BU[0] = rl(10.)

    By_left = rl(7.0)
    By_right = rl(-7.0)
    BU[1] = BU[2] = noif.coord_less_bound(x, bound)*By_left \
                  + noif.coord_geq_bound(x,bound)*By_right

    return rho, press, vU, BU

##### <a id='balsara5'></a>

## Step 2.e: Balsara 5 \[Back to [top](#toc)\]
$$\label{balsara5}$$

\begin{align}
\rho &= \left \{ \begin{array}{lll} 1.08 & \mbox{if} & x < 0 \\
1.0 & \mbox{if} & x \geq 0 \end{array} \right. \\
p &= \left \{ \begin{array}{lll} 0.95 & \mbox{if} & x < 0 \\
1.0 & \mbox{if} & x \geq 0 \end{array} \right. \\
v^x &= \left \{ \begin{array}{lll} 0.4 & \mbox{if} & x < 0 \\
-0.45 & \mbox{if} & x \geq 0 \end{array} \right. \\
v^y &= \left \{ \begin{array}{lll} 0.3 & \mbox{if} & x < 0 \\
-0.2 & \mbox{if} & x \geq 0 \end{array} \right. \\
v^z &= 0.2, \\
B^x &= 2.0, \\
B^y &= \left \{ \begin{array}{lll} 0.3 & \mbox{if} & x < 0 \\
-0.7 & \mbox{if} & x \geq 0 \end{array} \right. \\
B^z &= \left \{ \begin{array}{lll} 0.3 & \mbox{if} & x < 0 \\
0.5 & \mbox{if} & x \geq 0 \end{array} \right. \\
\end{align}

In [6]:
def balsara5(x, bound = 0.0):
    vU = ixp.zerorank1()
    BU = ixp.zerorank1()

    rho_left = rl(108,100)
    rho_right = rl(1.0)
    rho = noif.coord_less_bound(x, bound)*rho_left \
        + noif.coord_geq_bound(x,bound)*rho_right

    press_left = rl(95, 100)
    press_right = rl(1.0)
    press = noif.coord_less_bound(x, bound)*press_left \
          + noif.coord_geq_bound(x,bound)*press_right

    vx_left = rl(4,10)
    vx_right = rl(-45,100)
    vU[0] = noif.coord_less_bound(x, bound)*vx_left \
          + noif.coord_geq_bound(x,bound)*vx_right

    vy_left = rl(3,10)
    vy_right = rl(-2,10)
    vU[1] = noif.coord_less_bound(x, bound)*vy_left \
          + noif.coord_geq_bound(x,bound)*vy_right

    vU[2] = rl(2,10)

    BU[0] = rl(2.)

    By_left = rl(3,10)
    By_right = rl(-7,10)
    BU[1] = noif.coord_less_bound(x, bound)*By_left \
          + noif.coord_geq_bound(x,bound)*By_right

    Bz_left = rl(3,10)
    Bz_right = rl(5,10)
    BU[2] = noif.coord_less_bound(x, bound)*Bz_left \
          + noif.coord_geq_bound(x,bound)*Bz_right

    return rho, press, vU, BU

<a id='BtoA'></a>

# Step 3: Converting to Vector Potential $A_i$\[Back to [top](#toc)\]
$$\label{BtoA}$$

Some GRMHD codes elect to evolve the vector potential $A_i$ instead of the magnetic field $B^i$ directly, either to maintain the divergence-free nature of the magnetic fields across grid boundaries or throughout evolution. While we can convert from $A_i \rightarrow B^i$ using

$$
B^i = \epsilon^{ijk} \partial_j A_k,
$$

where $\epsilon^{ijk}$ is the Levi-Civita tensor, going from $B^i \rightarrow A_i$ is non-trivial for generic magnetic fields. However, given that the fields here are piece-wise constant, for Cartesian coordinates and flat spacetime we can therefore write

$$
A_i = \left( z B^y, x B^z, y B^x \right).
$$

Here we create a function to deal with implement the above.

In [7]:
def BtoA_piecewise_constant_Cart_Flat(x, y, z, BU):
    AD = ixp.zerorank1()
    AD[0] = z*BU[1]
    AD[1] = x*BU[2]
    AD[2] = y*BU[0]

    return AD

<a id='mrhs'></a>

# Step 4: Mildly Relativistic Hydrodynamic Shock \[Back to [top](#toc)\]
$$\label{mrhs}$$

Here we define initial data for pure hydrodynamics, i.e. $B^i = 0$. In this section we construct test problem 1 from [Marti and Muller (2003)](https://link.springer.com/article/10.12942/lrr-2003-7), table 7. With $v^i = 0$ everywhere, we define the density and pressure profiles as

\begin{align}
\rho &= \left \{ \begin{array}{lll} 10.0 & \mbox{if} & x < 0.5 \\
1.0 & \mbox{if} & x \geq 0.5 \end{array} \right. \\
p &= \left \{ \begin{array}{lll} 40.0/3.0 & \mbox{if} & x < 0.5 \\
0.0 & \mbox{if} & x \geq 0.5 \end{array} \right. \\
\end{align}

In [8]:
def hydro1(x, bound = 0.5):
    vU = ixp.zerorank1()
    BU = ixp.zerorank1()

    rho_left = rl(10.)
    rho_right = rl(1.0)
    rho = noif.coord_less_bound(x, bound)*rho_left \
        + noif.coord_geq_bound(x,bound)*rho_right

    press_left = rl(40, 3)
    press_right = rl(0.0)
    press = noif.coord_less_bound(x, bound)*press_left \
          + noif.coord_geq_bound(x,bound)*press_right

    return rho, press, vU, BU

<a id='rhbw'></a>

# Step 5: Relativistic Hydrodynamic Blast Wave \[Back to [top](#toc)\]
$$\label{rhbw}$$

Similar to the previous section, here we construct test problem 2 from [Marti and Muller (2003)](https://link.springer.com/article/10.12942/lrr-2003-7), table 7. Again with $v^i = 0$ everywhere, we define the density and pressure profiles as

\begin{align}
\rho &= 1.0 \\
p &= \left \{ \begin{array}{lll} 1000.0 & \mbox{if} & x < 0.5 \\
0.01 & \mbox{if} & x \geq 0.5 \end{array} \right. \\
\end{align}

In [9]:
def hydro2(x, bound = 0.5):
    vU = ixp.zerorank1()
    BU = ixp.zerorank1()

    rho = rl(1.0)

    press_left = rl(1000.0)
    press_right = rl(1, 100)
    press = noif.coord_less_bound(x, bound)*press_left \
          + noif.coord_geq_bound(x,bound)*press_right

    return rho, press, vU, BU

<a id='code_validation'></a>

# Step 4: Code Validation against `ShockTests_1D` NRPy+ Module \[Back to [top](#toc)\]
$$\label{code_validation}$$

Here, as a code validation check, we verify agreement in the SymPy expressions for the 1D shock tests we intend to use between
1. this tutorial and 
2. the NRPy+ [`ShockTests_1D.py`](../../edit/in_progress-ShockTests/ShockTests_1D.py) module.


In [10]:
# here we modify the check_zero function
from UnitTesting.assert_equal import check_zero

def assert_zero(exp):
    assert check_zero(exp)==True

def compare_results(functionA, functionB, x, bound=0.0):
    rho_A, press_A, vU_A, BU_A = functionA(x, bound)
    rho_B, press_B, vU_B, BU_B = functionB(x, bound)

    assert_zero(rho_A - rho_B)
    assert_zero(press_A - press_B)

    for i in range(DIM):
        assert_zero(vU_A[i] - vU_B[i])
        assert_zero(BU_A[i] - BU_B[i])

In [11]:
import ShockTests_1D as st_1d

x = rfm.xx_to_Cart[0]
y = rfm.xx_to_Cart[1]
z = rfm.xx_to_Cart[2]

compare_results(balsara1, st_1d.balsara1, x)
compare_results(balsara2, st_1d.balsara2, z)
compare_results(balsara3, st_1d.balsara3, y)
compare_results(balsara4, st_1d.balsara4, z)
compare_results(balsara5, st_1d.balsara5, y)
compare_results(hydro1, st_1d.hydro1, z)
compare_results(hydro2, st_1d.hydro2, x)


_,_,_, BU = balsara5(x, bound=0.23)
AD_A = BtoA_piecewise_constant_Cart_Flat(x,y,z, BU)
AD_B = st_1d.BtoA_piecewise_constant_Cart_Flat(x,y,z, BU)

for i in range(DIM):
    assert_zero(AD_A[i] - AD_B[i])

print('All assertions have passed!')

All assertions have passed!


In [12]:
sqrtgammaDET = sp.sympify(1.0)
LeviCivitaTensorUUU = ixp.LeviCivitaTensorUUU_dim3_rank3(sqrtgammaDET)

_, _, _, BU_test = balsara5(y, bound=3.0)
AD_test = BtoA_piecewise_constant_Cart_Flat(rfm.Cartx, rfm.Carty, rfm.Cartz, BU_test)

BU_compare = ixp.zerorank1()
for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            BU_compare[i] += LeviCivitaTensorUUU[i][j][k]*sp.diff(AD_test[k], rfm.Cart[j])

for w in range(DIM):
    assert_zero(sp.simplify(BU_test[w] - BU_compare[w]))

print('BtoA passes!')

BtoA passes!


<a id='latex_pdf_output'></a>

# Step 5: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-GRMHD-1D_ShockTests.pdf](Tutorial-GRMHD-1D_ShockTests.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [13]:
import cmdline_helper as cmd    # NRPy+: Multi-platform Python command-line interface
cmd.output_Jupyter_notebook_to_LaTeXed_PDF("Tutorial-GRMHD-1D_ShockTests")

Created Tutorial-GRMHD-1D_ShockTests.tex, and compiled LaTeX file to PDF
    file Tutorial-GRMHD-1D_ShockTests.pdf
